## Select Assignment Number

To load the correct settings, the number for the current assignment is set. 

In [1]:
assignment_nr = 1

## Load Packages and Global Settings

In [2]:
# Package imports
from canvasapi import Canvas
from canvasapi.requester import Requester
from canvas_connector.utils.canvas_utils import download_assignment_submissions
from openai import OpenAI
import os
import zipfile
import shutil
import re

# Local imports
from scripts.jsonify import jsonify, jsonify_resources, analyze_jsonify_results
from scripts.utils import create_file_list

In [3]:
# Load global settings
from scripts.settings import *

# Load assignment specific settings
ASSIGNMENT = ASSIGNMENTS[assignment_nr]
ASSIGNMENT_ID = ASSIGNMENT["canvas"]["assignment_id"]
QUIZ_ID = ASSIGNMENT["canvas"]["quiz_id"]
R_QUIZ_QUESTION_ID = ASSIGNMENT["canvas"]["r_quiz_question_id"]
ADV_QUIZ_QUESTION_ID = ASSIGNMENT["canvas"]["adv_quiz_question_id"]
LOCK_GRADES_DATE = ASSIGNMENT["lock_grades_date"]

In [4]:
# Initialize Canvas API
canvas_client = Canvas(os.getenv("CANVAS_API_URL"), os.getenv("CANVAS_API_KEY"))
canvas_requester = Requester(os.getenv("CANVAS_API_URL"), os.getenv("CANVAS_API_KEY"))

# Initialize OpenAI API
if USE_UVA_OPENAI:
    openai_client = OpenAI(api_key=os.getenv("UVA_OPENAI_API_KEY"), 
                           base_url=os.getenv("UVA_OPENAI_BASE_URL"))
else:
    openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Jsonify Resources

To ensure the latest changes to rubrics, assignment, example solutions, or goals are captured, the resources are jsonified.

In [5]:
analyze_jsonify_results(jsonify_resources(assignment_nr, RESOURCES_PATH))

questions: 48 (R: 30, Radv: 8, Python: 10)
rubrics: 48 (R: 30, Radv: 8, Python: 10)
solutions: 48 (R: 30, Radv: 8, Python: 10)
goals: 48 (R: 30, Radv: 8, Python: 10)


## Download and Prepare Submissions

All assignment submissions are downloaded and jsonified.

In [14]:
# Download assignment submissions
user_whitelist = [84259]
user_blacklist = []
out_paths = download_assignment_submissions(canvas_requester, COURSE_ID, ASSIGNMENT_ID, user_whitelist, user_blacklist)

# Jsonify submissions
for out_path in out_paths:
    jsonify(out_path, ".".join(out_paths[0].split(".")[0:-1]) + ".json")